# 카카오 API를 활용하여 지오코딩 및 POI 데이터 수집하기

# 1. 카카오 API키 발급받기

API 키(Application Programming Interface Key)는 소프트웨어 간의 상호작용을 위해 사용되는 코드입니다. 개발자가 특정 API를 사용할 권한이 있음을 인증하는 역할을 합니다. API 키는 일반적으로 서비스 제공자가 사용자 또는 애플리케이션의 식별과 인증, API 사용량 제한, 보안 강화 등을 위해 발급합니다. API 키를 사용함으로써 서비스 제공자는 누가 API를 사용하고 있는지 추적할 수 있으며, 부적절한 사용을 방지할 수 있습니다.

카카오 API 키를 발급받으려면 다음 단계를 따르세요:

1. 카카오 개발자 웹사이트(https://developers.kakao.com/)에 접속하세요.
2. 카카오 계정으로 로그인하거나 새로운 계정을 만드세요.
3. 로그인 후, "내 애플리케이션"을 클릭하세요.
4. "애플리케이션 추가하기" 버튼을 클릭하세요.
5. 애플리케이션 이름을 입력하고, 사용할 플랫폼(웹, 앱 등)을 선택하세요.
6. 애플리케이션을 생성한 후, "API 키" 탭으로 이동하세요.
7. "REST API 키"를 확인하고, 해당 키를 복사하세요.
이제 카카오 API 키를 사용하여 카카오 API를 호출할 수 있습니다. 주의할 점은 API 키를 외부에 노출하지 않도록 보안에 유의해야 합니다.

참고: https://sorrow16.tistory.com/187

본 문서에서는 아래의 4가지 패키지를 사용합니다. 

`request`는 Python을 사용하여 HTTP 요청을 보내는 Python 패키지입니다. HTTP 요청을 보내고 웹에서 응답을 받을 수 있습니다. <br>
`json`은 Python에서 JSON 데이터를 다루는 데 사용되는 내장 패키지입니다. JSON(JavaScript Object Notation)은 데이터를 저장하고 교환하기 위한 경량의 데이터 형식입니다. <br>
`pandas`는 데이터 분석을 위한 Python 패키지입니다. 데이터를 효과적으로 처리하고 분석할 수 있습니다. <br>
`geopandas`는 지리 공간 데이터를 다루기 위한 Python 패키지입니다. 지리 공간 데이터를 시각화하고 분석할 수 있습니다. <br>

In [ ]:
import requests
import json
import geopandas as gpd
import pandas as pd
from string import Template

# 2. 경희대학교 주소 지오코딩하기

지오코딩(Geocoding)이란 주소, 지명, 우편번호 등과 같은 텍스트 기반의 위치 데이터를 위도(latitude)와 경도(longitude)와 같은 지리적 좌표로 변환하는 과정을 의미합니다. 

카카오 API에서는 지오코딩 API 사용법을 다음 웹사이트에서 확인할 수 있습니다: https://developers.kakao.com/docs/latest/ko/local/dev-guide#address-coord


In [ ]:
# 예시: 경희대학교 주소 지오코딩하기
# 검색할 주소: 서울특별시 동대문구 경희대로 26 
# 도출되는 경위도 (127.054890960564, 37.5939491407769)

# Kakao REST API 세팅
REST_API_KEY = "10a11b38e517203e094d147dae1789e0" # Kakao REST API 키
FORMAT = 'json' # 반환 형식: json 또는 xml
URL_geocoding = Template("https://dapi.kakao.com/v2/local/search/address.${FORMAT}").substitute(FORMAT=FORMAT) # 주소 검색 API

# API의 Header는 HTTP 요청 또는 응답 메시지의 일부로, 메시지의 본문(body)이 아닌, 데이터에 대한 메타데이터(예: 데이터 형식, 인증 정보 등)를 포함
headers = {'Authorization': Template('KakaoAK ${REST_API_KEY}').substitute(REST_API_KEY=REST_API_KEY)}

# 검색할 주소 입력
query_address = '서울특별시 동대문구 경희대로 26' 

# requests.get 함수는 Python의 requests 라이브러리를 사용하여 HTTP GET 요청을 서버에 보내는 역할
response = requests.get(URL_geocoding, headers=headers, params={'query': query_address})

# response.status_code는 서버의 응답 상태 코드를 반환 (200은 성공을 의미)
if response.status_code == 200:
    data = json.loads(response.text)
    
    print(f'''
          검색된 주소: {data['documents'][0]['address_name']}
          경도: {data['documents'][0]['x']}, 
          위도: {data['documents'][0]['y']}''')


## 2.1. REST API 검색 엔진 활성화

In [ ]:
FORMAT = 'json' # 반환 형식: json 또는 xml

# Kakao REST API 설명에 따라 URL 생성
URL_geocoding = Template("https://dapi.kakao.com/v2/local/search/address.${FORMAT}").substitute(FORMAT=FORMAT) # 주소 검색 API

# f-string을 사용하여 URL 생성 (위와 동일한 결과)
URL_geocoding_f = f"https://dapi.kakao.com/v2/local/search/address.{FORMAT}"

print(URL_geocoding)
print(URL_geocoding_f)

## 2.2. API 키 및 검색내용 입력 후 호출

In [ ]:
REST_API_KEY = '10a11b38e517203e094d147dae1789e0' # 'YOUR API KEY' # Paste your API key here 

# API의 Header는 HTTP 요청 또는 응답 메시지의 일부로, 메시지의 본문(body)이 아닌, 데이터에 대한 메타데이터(예: 데이터 형식, 인증 정보 등)를 포함
# headers = {'Authorization': Template('KakaoAK ${REST_API_KEY}').substitute(REST_API_KEY=REST_API_KEY)}

# f-string을 사용하여 API 키 헤더로 입력 (위와 동일한 결과)
headers = {'Authorization': f'KakaoAK {REST_API_KEY}'} 

# 검색할 주소 입력
query_address = '서울특별시 동대문구 경희대로 26' 

# requests.get 함수는 Python의 requests 라이브러리를 사용하여 HTTP GET 요청을 서버에 보내는 역할
response = requests.get(URL_geocoding, headers=headers, params={'query': query_address})
print(response.status_code)

## 2.3. API 결과 포맷팅

카카오 지오코딩 API의 경우, 아래의 결과 형식으로 반환됨 <br>
https://developers.kakao.com/docs/latest/ko/local/dev-guide#address-coord-response-body-document

JSON을 활용하여 결과를 파싱하고, 필요한 정보만 추출하여 사용할 수 있음.

In [ ]:
# response.text 속성은 서버 응답의 본문을 반환
print(response.text)

In [ ]:
# response.json() 함수는 서버 응답의 본문을 JSON 형식으로 반환
# response.json() 함수를 사용하여 API 결과를 data 변수에 저장
data = response.json()
print(data)

In [ ]:
# json.dumps() 함수는 Python 객체를 JSON 형식의 문자열로 변환 및 포맷팅
# API 마다 반환되는 JSON 형식이 다르므로, JSON 결과를 보기 쉽게 출력하기 위해 사용
# JSON은 쉽게 생각해서 Python의 딕셔너리와 유사한 형태

print(json.dumps(data, indent=2, ensure_ascii=False))

In [ ]:
# 위도 및 경도 추출
print(f"경도: {data['documents'][0]['x']}")
print(f"위도: {data['documents'][0]['y']}")

### *Exercise*

아래의 코드를 활용하여, 경희대학교 국제캠퍼스의 주소를 검색하세요. 

```python
YOUR_API_KEY = 'YOUR API KEY' # Paste your API key here

# Kakao REST API 설명에 따라 URL 생성
FORMAT = 'json' # 반환 형식: json 또는 xml
temp_URL = f"https://dapi.kakao.com/v2/local/search/address.{FORMAT}"

# f-string을 사용하여 API 키 헤더로 입력 
temp_headers = {'Authorization': f'KakaoAK {`API 입력`}'} 

# 검색할 주소 입력 (경기도 용인시 기흥구 덕영대로 1732)
temp_query_address = `검색할 주소 입력`

# requests.get을 활용하여 API 요청
response = requests.get(`URL 입력`, headers=`헤더 (API 키) 입력`, params={'query': `검색할 주소 입력`})

print(response.status_code)
```

In [ ]:
# Your code here

YOUR_API_KEY = 'YOUR API KEY' # Paste your API key here

# Kakao REST API 설명에 따라 URL 생성
FORMAT = 'json' # 반환 형식: json 또는 xml
temp_URL = f"https://dapi.kakao.com/v2/local/search/address.{FORMAT}"

# f-string을 사용하여 API 키 헤더로 입력 
temp_headers = {'Authorization': f'KakaoAK {`API 입력`}'} 

# 검색할 주소 입력 (경기도 용인시 기흥구 덕영대로 1732)
temp_query_address = `검색할 주소 입력`

# requests.get을 활용하여 API 요청
response = requests.get(`URL 입력`, headers=`헤더 (API 키) 입력`, params={'query': `검색할 주소 입력`})

print(response.status_code)

In [ ]:
# 아래의 코드가 정상적으로 작동하면, Success가 출력됨

assert temp_URL == 'https://dapi.kakao.com/v2/local/search/address.json'
assert type(temp_headers) == dict
assert temp_headers['Authorization'][0:7] == 'KakaoAK'
assert temp_query_address == '경기도 용인시 기흥구 덕영대로 1732'
assert response.status_code == 200

print('Success')

### *Exercise*

아래의 코드를 활용하여, 검색된 API에서 경위도를 추출하여, `temp_lat` (위도), `temp_lng` (경도) 변수에 할당하세요. 

```python
# 검색된 API를 JSON 형식으로 변환
temp_data = response.json()

# 딕셔너리를 슬라이스하여 위도 및 경도 추출
# 힌트: https://developers.kakao.com/docs/latest/ko/local/dev-guide#address-coord-response-body-document
temp_lat = `위도 추출` # 위도
temp_lng = `경도 추출` # 경도

print(temp_lat, temp_lng)
```


In [ ]:
# Your code here

# 검색된 API를 JSON 형식으로 변환
temp_data = response.json()

# 딕셔너리를 슬라이스하여 위도 및 경도 추출
# 힌트: https://developers.kakao.com/docs/latest/ko/local/dev-guide#address-coord-response-body-document
temp_lat = `위도 추출` # 위도
temp_lng = `경도 추출` # 경도

print(temp_lat, temp_lng)

In [ ]:
# 아래의 코드가 정상적으로 작동하면, Success가 출력됨
assert round(float(temp_lat), 4) == 37.2398
assert round(float(temp_lng), 4) == 127.0812

print('Success')

# 3. 키워드로 POI 검색하기

카카오 API는 키워드로 POI(Point of Interest)를 검색할 수 있는 기능을 제공합니다. POI는 주변에 있는 관심 지점을 의미하며, 주변에 있는 음식점, 카페, 병원, 학교 등을 검색할 수 있습니다.

참고: https://developers.kakao.com/docs/latest/ko/local/dev-guide#search-by-keyword



## 3.1. REST API를 활용하여 POI 검색

In [ ]:
FORMAT = "json"
URL_poi = f"https://dapi.kakao.com/v2/local/search/keyword.{FORMAT}" 
# REST_API_KEY = YOUR_API_KEY

headers = {'Authorization': f'KakaoAK {REST_API_KEY}'}
query_keyword = '편의점'

response_poi = requests.get(URL_poi, 
                        headers=headers, 
                        params={'query': query_keyword, # 검색 키워드
                                'x': '127.054890960564', # 검색 중심 좌표 (경도)
                                'y': '37.5939491407769', # 검색 중심 좌표 (위도)
                                'radius': 500 # 검색 반경 500m
                                }
                                )

response_poi.status_code

## 3.2. 검색 결과 포맷팅

In [ ]:
# 검색된 API를 JSON 형식으로 변환
data_poi = response_poi.json()

# JSON 결과를 보기 쉽게 출력
print(json.dumps(data_poi, indent=2, ensure_ascii=False))

## 3.3. 검색 결과 DataFrame으로 변환

In [ ]:
poi_df = pd.DataFrame(data=data_poi['documents']) # 데이터프레임 변환
poi_df

## 3.4. DataFrame의 X, Y 변수를 활용하여 GeoDataFrame으로 변환

In [ ]:
# 지오데이터프레임 변환
poi_gdf = gpd.GeoDataFrame(data=poi_df, 
                           geometry=gpd.points_from_xy(poi_df['x'], poi_df['y']), # ArcGIS의 XY Table to Point와 유사한 역할
                           crs='EPSG:4326' # 좌표계 설정 (WGS84)
                           ) 
poi_gdf

In [ ]:
# 동적인 지도로 결과 출력
poi_gdf.explore()

### *Exercise*

아래의 코드를 활용하여, 원하는 위치에 원하는 키워드로 POI를 검색하세요. Status_code가 200이면 성공적으로 API가 호출되었음을 의미합니다. <br>
참고: https://developers.kakao.com/docs/latest/ko/local/dev-guide#search-by-keyword


```python
FORMAT = "json"
URL_poi = f"https://dapi.kakao.com/v2/local/search/keyword.{FORMAT}" 

# 헤더 설정
headers = {'Authorization': f'KakaoAK {REST_API_KEY}'}

# 검색 키워드 설정
query_keyword = `Your Keyword`

# API 호출
your_response = requests.get(URL_poi, 
                             headers=headers, 
                             params={'query': query_keyword, # 검색 키워드
                                     'x': `YOUR Longitude`, # 검색 중심 좌표 (경도)
                                     'y': `Your Latitude`, # 검색 중심 좌표 (위도)
                                     'radius': `Your Search Width` # 검색 반경 
                                    }
                         )

# API 상태 확인
print(your_response.status_code)

```

In [ ]:
# Your code here
FORMAT = "json"
URL_poi = f"https://dapi.kakao.com/v2/local/search/keyword.{FORMAT}" 

# 헤더 설정
headers = {'Authorization': f'KakaoAK {REST_API_KEY}'}

# 검색 키워드 설정
query_keyword = `Your Keyword`

# API 호출
your_response = requests.get(URL_poi, 
                             headers=headers, 
                             params={'query': query_keyword, # 검색 키워드
                                     'x': `YOUR Longitude`, # 검색 중심 좌표 (경도)
                                     'y': `Your Latitude`, # 검색 중심 좌표 (위도)
                                     'radius': `Your Search Width` # 검색 반경 
                                    }
                         )

# API 상태 확인
print(your_response.status_code)


### *Exercise*

아래의 코드를 활용하여, 호출된 API를 JSON 형식으로 변환 -> DataFrame으로 변환 -> GeoDataFrame으로 변환하세요. 동적인 지도가 생성되며, 검색된 POI를 확인할 수 있습니다.

```python
# 검색된 API를 JSON 형식으로 변환
your_poi = `YOUR API RRESPONSE`.json()

# 데이터프레임 변환
your_df = pd.DataFrame(data=`JSON 형식의 변수`) 

# 지오데이터프레임 변환
your_gdf = gpd.GeoDataFrame(data=`데이터 프레임 변수`, 
                           geometry=gpd.points_from_xy(`데이터 프레임의 경도 Column, 데이터 프레임의 위도 Column`), # ArcGIS의 XY Table to Point와 유사한 역할
                           crs=`EPSG:EPSG 코드` # 좌표계 설정 (WGS84)
                           ) 

# 동적 지도로 결과 확인
your_gdf.explore()
```

In [ ]:
# Your code here

# 검색된 API를 JSON 형식으로 변환
your_poi = `YOUR API RRESPONSE`.json()

# 데이터프레임 변환
your_df = pd.DataFrame(data=`JSON 형식의 변수`) 

# 지오데이터프레임 변환
your_gdf = gpd.GeoDataFrame(data=`데이터 프레임 변수`, 
                           geometry=gpd.points_from_xy(`데이터 프레임의 경도 Column, 데이터 프레임의 위도 Column`), # ArcGIS의 XY Table to Point와 유사한 역할
                           crs=`EPSG 코드` # 좌표계 설정 (WGS84)
                           ) 
# 동적 지도로 결과 확인
your_gdf.explore()

# 4. 고급 기능
## 4.1. 여러 페이지의 POI를 검색

In [ ]:
# 검색된 API를 JSON 형식으로 변환
data_poi = response_poi.json()

# JSON 결과를 보기 쉽게 출력
print(json.dumps(data_poi, indent=2, ensure_ascii=False))

In [ ]:
# 아래의 변수를 통하여 총 몇 개의 편의점이 검색되었는지 확인
# 47개의 편의점이 검색되어야 하지만, 현재는 15개의 편의점만 검색됨
data_poi['meta']['total_count']

In [ ]:
# is_end 키를 통해 마지막 페이지인지 확인
# 마지막 페이지가 아니라면 다음 페이지에서 더 많은 편의점을 불러올 수 있음
data_poi['meta']

In [ ]:
# While 구문을 활용하여 모든 편의점 검색 (최대 45개)

FORMAT = "json"
URL_poi = f"https://dapi.kakao.com/v2/local/search/keyword.{FORMAT}" 
# REST_API_KEY = YOUR_API_KEY

headers = {'Authorization': f'KakaoAK {REST_API_KEY}'}
query_keyword = '편의점'

# 출력을 저장할 DataFrame 생성
result_df = pd.DataFrame()

# While 반복문을 통해 API 호출
_status = False 
_page_inst = 1 # API 호출 페이지 수

while _status is False:

    response_poi = requests.get(URL_poi, 
                            headers=headers, 
                            params={'query': query_keyword, # 검색 키워드
                                    'x': '127.054890960564', # 검색 중심 좌표 (경도)
                                    'y': '37.5939491407769', # 검색 중심 좌표 (위도)
                                    'radius': 500, # 검색 반경 500m
                                    'page': _page_inst # API 호출 페이지 수
                                    }
                                    )
    print(f"Page {_page_inst} 검색 완료")

    # 페이지 인스턴스 증가 (다음 페이지 호출)
    _page_inst += 1

    # 기존의 DataFrame에 새로운 결과를 추가
    result_df = pd.concat([result_df, 
                           pd.DataFrame(response_poi.json()['documents'])],
                             axis=0).reset_index(drop=True)
    
    # 만약 마지막 페이지라면 True가 반환되며, While 반복문 종료
    _status = response_poi.json()['meta']['is_end']

result_df

In [ ]:
# GeoDataFrame 변환
result_gdf = gpd.GeoDataFrame(result_df, 
                 geometry=gpd.points_from_xy(result_df['x'], result_df['y']),
                 crs='EPSG:4326')
result_gdf

In [ ]:
# 동적인 지도로 결과 출력
result_gdf.explore()

## 4.2. API 검색을 함수로 만들기

In [ ]:
def retrieve_POI_KakaoAPI(_API_KEY, _POI_NAME, _ORIGIN_COORDINATES, _RADIUS):
    '''
    _API_KEY: (String) Kakao API Key
    _POI_NAME: (String) Name of the POI to search for
    _ORIGIN_COORDINATES: (Tuple/List) Coordinates of the origin point (longitude, latitude)
    _RADIUS: (Int/Float) Search radius in meters
    '''
    
    # Define the URL, headers, and parameters
    URL = f"https://dapi.kakao.com/v2/local/search/keyword.json"
    headers = {'Authorization': f'KakaoAK {_API_KEY}'}
    params = {'query': _POI_NAME,
              'x': _ORIGIN_COORDINATES[0], # 검색 중심 좌표 (경도)
              'y': _ORIGIN_COORDINATES[1], # 검색 중심 좌표 (위도)
              'radius': _RADIUS, # 검색 반경 (meters)
              }

    response = requests.get(URL, headers=headers, params=params)
    
    # 검색이 잘 되었을 경우
    if response.status_code == 200:
        print("POI DATA RETRIEVED SUCCESSFULLY")

        # Convert the response to JSON
        response_json = response.json()
        
        # # Convert the JSON to a DataFrame
        response_df = pd.DataFrame(response_json['documents'])

        # Convert the DataFrame to a GeoDataFrame
        response_gdf = gpd.GeoDataFrame(response_df,
                                        geometry=gpd.points_from_xy(response_df['x'], response_df['y']),
                                        crs='EPSG:4326')           

        return response_gdf
    
    # 검색이 잘 되지 않았을 경우
    else:
        print("ERROR: POI DATA RETRIEVAL FAILED")
        return response.text



In [ ]:
REST_API_KEY = "10a11b38e517203e094d147dae1789e0" # Kakao REST API 키
POI_NAME = '병원' 
ORIGIN_COORDINATES = (127.054890960564, 37.5939491407769) # 경희대학교 좌표
RADIUS = 5000 # 500m 반경 내 편의점 검색

test_def = retrieve_POI_KakaoAPI(REST_API_KEY, POI_NAME, ORIGIN_COORDINATES, RADIUS)
test_def

In [ ]:
test_def.explore()

# 끝